In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 200
lr = 0.04
schedule = [200, 1500, 3000, 4500, 6000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_star/cutmix' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.01
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]

# cutmix
cm_prob = 1
cm_prob_init = 0.99
cm_prob_low = 0.01

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index]
        tt = targets[rand_index]
        rand_index = rand_index[st == tt]
#         print(rand_index)

        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(source_train_loader, train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 5000] LR: 0.040000
1/7 Data:10.664 | Batch:26.459 | Total:0:00:21 | ETA:0:02:07 | Loss:0.001191709190607071 | top1:100.0
2/7 Data:0.002 | Batch:0.745 | Total:0:00:21 | ETA:0:00:55 | Loss:0.0007837748853489757 | top1:100.0
3/7 Data:0.016 | Batch:0.462 | Total:0:00:22 | ETA:0:00:30 | Loss:0.0018383982436110575 | top1:100.0
4/7 Data:0.015 | Batch:0.414 | Total:0:00:22 | ETA:0:00:18 | Loss:0.0015751559112686664 | top1:100.0
5/7 Data:0.015 | Batch:0.391 | Total:0:00:23 | ETA:0:00:10 | Loss:0.001428803673479706 | top1:100.0
6/7 Data:0.018 | Batch:0.395 | Total:0:00:23 | ETA:0:00:04 | Loss:0.004510213824687526 | top1:100.0


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


153/153 Data:0.000 | Batch:3.182 | Total:0:02:22 | ETA:0:00:00 | Loss:4.365136923046287 | top1:50.00655746459961
39/39 Data:0.005 | Batch:0.490 | Total:0:02:33 | ETA:0:00:00 | Loss:0.0074400490176720685 | top1:99.73076629638672

Epoch: [2 | 5000] LR: 0.042400
1/7 Data:0.936 | Batch:1.401 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0006587743409909308 | top1:100.0
2/7 Data:0.022 | Batch:0.460 | Total:0:00:01 | ETA:0:00:04 | Loss:0.003116637730272487 | top1:100.0
3/7 Data:0.021 | Batch:0.485 | Total:0:00:01 | ETA:0:00:03 | Loss:0.0022913889454988143 | top1:100.0
4/7 Data:0.019 | Batch:0.468 | Total:0:00:02 | ETA:0:00:02 | Loss:0.008321683955728076 | top1:100.0
5/7 Data:0.015 | Batch:0.460 | Total:0:00:02 | ETA:0:00:02 | Loss:0.007021142158191651 | top1:100.0
6/7 Data:0.018 | Batch:0.454 | Total:0:00:03 | ETA:0:00:01 | Loss:0.006731923271824296 | top1:100.0

Epoch: [3 | 5000] LR: 0.044800
1/7 Data:0.976 | Batch:1.440 | Total:0:00:00 | ETA:0:00:06 | Loss:0.08743947744369507 | top1:96.875
2/7 Da

4/7 Data:0.001 | Batch:4.083 | Total:0:00:11 | ETA:0:00:09 | Loss:249.55177688598633 | top1:98.4375
5/7 Data:0.001 | Batch:4.918 | Total:0:00:16 | ETA:0:00:07 | Loss:250.104833984375 | top1:97.5
6/7 Data:0.002 | Batch:1.537 | Total:0:00:18 | ETA:0:00:04 | Loss:250.52536265055338 | top1:96.35417175292969

Epoch: [15 | 5000] LR: 0.073600
1/7 Data:11.810 | Batch:15.181 | Total:0:00:09 | ETA:0:00:59 | Loss:252.210205078125 | top1:100.0
2/7 Data:0.006 | Batch:4.052 | Total:0:00:13 | ETA:0:00:35 | Loss:252.00045013427734 | top1:96.875
3/7 Data:0.070 | Batch:3.763 | Total:0:00:17 | ETA:0:00:24 | Loss:251.60555013020834 | top1:97.91667175292969
4/7 Data:0.001 | Batch:3.514 | Total:0:00:21 | ETA:0:00:16 | Loss:251.19403076171875 | top1:95.3125
5/7 Data:0.018 | Batch:4.284 | Total:0:00:25 | ETA:0:00:11 | Loss:250.68380126953124 | top1:93.125
6/7 Data:0.002 | Batch:4.254 | Total:0:00:29 | ETA:0:00:05 | Loss:250.0751698811849 | top1:92.1875

Epoch: [16 | 5000] LR: 0.076000
1/7 Data:13.928 | Batch:

1/7 Data:1.016 | Batch:1.532 | Total:0:00:01 | ETA:0:00:07 | Loss:1240.4185791015625 | top1:65.625
2/7 Data:0.027 | Batch:0.561 | Total:0:00:01 | ETA:0:00:04 | Loss:1233.7568359375 | top1:73.4375
3/7 Data:0.025 | Batch:0.656 | Total:0:00:02 | ETA:0:00:03 | Loss:1226.9002278645833 | top1:71.875
4/7 Data:0.023 | Batch:0.604 | Total:0:00:02 | ETA:0:00:03 | Loss:1219.6434631347656 | top1:74.21875
5/7 Data:0.018 | Batch:0.543 | Total:0:00:03 | ETA:0:00:02 | Loss:1212.0678466796876 | top1:76.875
6/7 Data:0.018 | Batch:0.565 | Total:0:00:03 | ETA:0:00:01 | Loss:1204.302490234375 | top1:74.47917175292969

Epoch: [28 | 5000] LR: 0.104800
1/7 Data:0.805 | Batch:1.324 | Total:0:00:00 | ETA:0:00:06 | Loss:1147.7960205078125 | top1:84.375
2/7 Data:0.015 | Batch:0.518 | Total:0:00:01 | ETA:0:00:04 | Loss:1138.6821899414062 | top1:84.375
3/7 Data:0.022 | Batch:0.623 | Total:0:00:02 | ETA:0:00:03 | Loss:1129.4875081380208 | top1:85.41667175292969
4/7 Data:0.019 | Batch:0.624 | Total:0:00:02 | ETA:0:00

...

2/7 Data:0.001 | Batch:0.383 | Total:0:00:01 | ETA:0:00:05 | Loss:0.0012459844583645463 | top1:100.0
3/7 Data:0.002 | Batch:0.468 | Total:0:00:02 | ETA:0:00:03 | Loss:0.008539579575881362 | top1:100.0
4/7 Data:0.004 | Batch:0.335 | Total:0:00:02 | ETA:0:00:02 | Loss:0.011816941958386451 | top1:100.0
5/7 Data:0.010 | Batch:0.396 | Total:0:00:02 | ETA:0:00:02 | Loss:0.035044277319684625 | top1:99.375
6/7 Data:0.001 | Batch:0.547 | Total:0:00:03 | ETA:0:00:01 | Loss:0.029520180522619437 | top1:99.47917175292969

Epoch: [4949 | 5000] LR: 0.000000
1/7 Data:1.383 | Batch:1.800 | Total:0:00:01 | ETA:0:00:07 | Loss:0.13522131741046906 | top1:93.75
2/7 Data:0.013 | Batch:0.378 | Total:0:00:01 | ETA:0:00:04 | Loss:0.07532810838893056 | top1:96.875
3/7 Data:0.001 | Batch:0.384 | Total:0:00:01 | ETA:0:00:03 | Loss:0.11517122170577447 | top1:96.875
4/7 Data:0.001 | Batch:0.515 | Total:0:00:02 | ETA:0:00:02 | Loss:0.10174992145039141 | top1:96.875
5/7 Data:0.001 | Batch:0.306 | Total:0:00:02 | ETA:0

1/7 Data:1.546 | Batch:2.150 | Total:0:00:01 | ETA:0:00:09 | Loss:0.00413172272965312 | top1:100.0
2/7 Data:0.001 | Batch:0.424 | Total:0:00:01 | ETA:0:00:05 | Loss:0.06194590195082128 | top1:98.4375
3/7 Data:0.001 | Batch:0.407 | Total:0:00:02 | ETA:0:00:04 | Loss:0.041630887504046164 | top1:98.95833587646484
4/7 Data:0.002 | Batch:0.448 | Total:0:00:02 | ETA:0:00:03 | Loss:0.03264030226273462 | top1:99.21875
5/7 Data:0.001 | Batch:0.416 | Total:0:00:03 | ETA:0:00:02 | Loss:0.02674264358356595 | top1:99.375
6/7 Data:0.000 | Batch:0.367 | Total:0:00:03 | ETA:0:00:01 | Loss:0.030496105318889022 | top1:98.95833587646484
153/153 Data:0.003 | Batch:0.163 | Total:0:00:40 | ETA:0:00:00 | Loss:2.200762253288829 | top1:50.19659423828125
39/39 Data:0.002 | Batch:0.195 | Total:0:00:15 | ETA:0:00:00 | Loss:0.19160261922157729 | top1:93.39743041992188

Epoch: [4962 | 5000] LR: 0.000000
1/7 Data:1.446 | Batch:1.933 | Total:0:00:01 | ETA:0:00:07 | Loss:0.011899027973413467 | top1:100.0
2/7 Data:0.01

4/7 Data:0.006 | Batch:0.523 | Total:0:00:02 | ETA:0:00:02 | Loss:0.08236872075940482 | top1:96.875
5/7 Data:0.001 | Batch:0.438 | Total:0:00:02 | ETA:0:00:02 | Loss:0.16012867426034064 | top1:93.75
6/7 Data:0.000 | Batch:0.334 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17303975948986286 | top1:93.75

Epoch: [4974 | 5000] LR: 0.000000
1/7 Data:1.717 | Batch:2.051 | Total:0:00:01 | ETA:0:00:07 | Loss:0.004775889217853546 | top1:100.0
2/7 Data:0.001 | Batch:0.271 | Total:0:00:01 | ETA:0:00:04 | Loss:0.10565954819321632 | top1:95.3125
3/7 Data:0.000 | Batch:0.359 | Total:0:00:01 | ETA:0:00:03 | Loss:0.07065139458669971 | top1:96.875
4/7 Data:0.001 | Batch:0.280 | Total:0:00:02 | ETA:0:00:02 | Loss:0.05380951604456641 | top1:97.65625
5/7 Data:0.002 | Batch:0.399 | Total:0:00:02 | ETA:0:00:01 | Loss:0.04339808514341712 | top1:98.125
6/7 Data:0.001 | Batch:0.461 | Total:0:00:02 | ETA:0:00:01 | Loss:0.03639872983330861 | top1:98.4375

Epoch: [4975 | 5000] LR: 0.000000
1/7 Data:1.482 | Batch:1.987


Epoch: [4986 | 5000] LR: 0.000000
1/7 Data:2.078 | Batch:2.802 | Total:0:00:01 | ETA:0:00:11 | Loss:0.018278520554304123 | top1:100.0
2/7 Data:0.001 | Batch:0.497 | Total:0:00:02 | ETA:0:00:06 | Loss:0.01451250072568655 | top1:100.0
3/7 Data:0.003 | Batch:0.437 | Total:0:00:02 | ETA:0:00:04 | Loss:0.011798815801739693 | top1:100.0
4/7 Data:0.010 | Batch:0.516 | Total:0:00:03 | ETA:0:00:03 | Loss:0.02549645071849227 | top1:99.21875
5/7 Data:0.001 | Batch:0.463 | Total:0:00:03 | ETA:0:00:02 | Loss:0.11384293846786023 | top1:96.25
6/7 Data:0.000 | Batch:0.653 | Total:0:00:04 | ETA:0:00:01 | Loss:0.0966039855654041 | top1:96.875

Epoch: [4987 | 5000] LR: 0.000000
1/7 Data:1.733 | Batch:2.303 | Total:0:00:01 | ETA:0:00:09 | Loss:0.09720627218484879 | top1:96.875
2/7 Data:0.004 | Batch:0.529 | Total:0:00:01 | ETA:0:00:05 | Loss:0.05075966566801071 | top1:98.4375
3/7 Data:0.006 | Batch:0.281 | Total:0:00:02 | ETA:0:00:03 | Loss:0.1196668694416682 | top1:96.875
4/7 Data:0.001 | Batch:0.348 | 

5/7 Data:0.005 | Batch:0.439 | Total:0:00:02 | ETA:0:00:02 | Loss:0.11716871857643127 | top1:95.0
6/7 Data:0.000 | Batch:0.391 | Total:0:00:03 | ETA:0:00:01 | Loss:0.10571152344346046 | top1:95.3125

Epoch: [4999 | 5000] LR: 0.000000
1/7 Data:1.147 | Batch:1.550 | Total:0:00:00 | ETA:0:00:06 | Loss:0.02245774306356907 | top1:100.0
2/7 Data:0.001 | Batch:0.460 | Total:0:00:01 | ETA:0:00:04 | Loss:0.022119784727692604 | top1:100.0
3/7 Data:0.016 | Batch:0.632 | Total:0:00:02 | ETA:0:00:03 | Loss:0.052037702252467476 | top1:97.91667175292969
4/7 Data:0.001 | Batch:0.359 | Total:0:00:02 | ETA:0:00:02 | Loss:0.046639939304441214 | top1:98.4375
5/7 Data:0.001 | Batch:0.540 | Total:0:00:02 | ETA:0:00:02 | Loss:0.0381102804094553 | top1:98.75
6/7 Data:0.005 | Batch:0.385 | Total:0:00:03 | ETA:0:00:01 | Loss:0.033078688817719616 | top1:98.95833587646484

Epoch: [5000 | 5000] LR: 0.000000
1/7 Data:1.343 | Batch:1.846 | Total:0:00:01 | ETA:0:00:07 | Loss:0.0017195381224155426 | top1:100.0
2/7 Dat